In [1]:
import pandas as pd
import os
import re
path = r"F:\myresearch\MIMICIIIsupport"
os.chdir(path)

In [106]:
d_icd_dia = pd.read_csv("D_ICD_DIAGNOSES.csv")
admission = pd.read_csv("admission_adult.csv",parse_dates=[4,5,6,15,16,22,23,24,25])
dias = pd.read_csv("DIAGNOSES_ICD.csv")

In [3]:
def GetComorbidities(comorbidity_codes,comorbidity_column_name):
    
    """
    抽取合并症数据
    comorbidity_codes:list or array or pd.Seires,根据github MIMIC代码获得,每个合并症需单独处理
    comorbidity_column_name:str,并入总表的列名，参照github代码取简称,如"chf"
    """
    comor = pd.DataFrame(columns=["HADM_ID",comorbidity_column_name])

    hadm_ids = dias["HADM_ID"].drop_duplicates()

    for hadm_id in hadm_ids:

        #print(hadm_id)
        df = pd.DataFrame()
        
        df.loc[0,"HADM_ID"] = hadm_id


        hadm_dias = dias[dias["HADM_ID"] == hadm_id]["ICD9_CODE"]

        contain =  1.in(comorbidity_codes.isin(hadm_dias).map({False:0,True:1}).values)

        #print(contain)
        
        if contain == True:
            df.loc[0,comorbidity_column_name] = 1
        else:        
            df.loc[0,comorbidity_column_name] = 0
        
        comor = comor.append(df)
        #comorbidity_table = pd.merge(admission["HADM_ID"],comor,on="HADM_ID") 
    return comor

In [153]:
##充血性心衰
chf_codes = d_icd_dia[(d_icd_dia.ICD9_CODE.isin(['39891','40201','40211','40291','40401','40403','40411','40413','40491','40493'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['4254','4255','4257','4258','4259'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['428']))]["ICD9_CODE"]## 心律失常
arrhy_codes = d_icd_dia[(d_icd_dia.ICD9_CODE.isin(['42613','42610','42612','99601','99604'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['4260','4267','4269','4270','4271','4272','4273','4274','4276','4278','4279','7850','V450','V533']))]["ICD9_CODE"]

##瓣膜病
valve_codes = d_icd_dia[
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['0932','7463','7464','7465','7466','V422','V433'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['394','395','396','397','424']))]["ICD9_CODE"]

## 肺循环障碍
pulmcirc_codes = d_icd_dia[#(d_icd_dia.ICD9_CODE.isin(['39891','40201','40211','40291','40401','40403','40411','40413','40491','40493'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['4150','4151','4170','4178','4179'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['416']))]["ICD9_CODE"]

## 周围血管病
perivasc_codes = d_icd_dia[#(d_icd_dia.ICD9_CODE.isin(['39891','40201','40211','40291','40401','40403','40411','40413','40491','40493'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['0930','4373','4431','4432','4438','4439','4471','5571','5579','V434'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['440','441']))]["ICD9_CODE"]

## 简单高血压
htn_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['401']))]["ICD9_CODE"]

##复杂高血压
htncx_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['402','403','404','405']))]["ICD9_CODE"]

##瘫痪
para_codes = d_icd_dia[#(d_icd_dia.ICD9_CODE.isin(['39891','40201','40211','40291','40401','40403','40411','40413','40491','40493'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['3341','3440','3441','3442','3443','3444','3445','3446','3449'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['342','343']))]["ICD9_CODE"]

##其他神经疾病
neuro_codes = d_icd_dia[#(d_icd_dia.ICD9_CODE.isin(['39891','40201','40211','40291','40401','40403','40411','40413','40491','40493'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['3319','3320','3321','3334','3335','3362','3481','3483','7803','7843'])) |
          (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['334','335','340','341','345']))]["ICD9_CODE"]

## COPD
chrnlung_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['4168','4169','5064','5081','5088'])) |
                           (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['490','491','492','493','494','495','496','500','501','502','503','504','505']))]["ICD9_CODE"]

##糖尿病
dm_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2500','2501','2502','2503']))]["ICD9_CODE"]

## 糖尿病慢性并发症
dmcx_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2504','2505','2506','2507','2508','2509']))]["ICD9_CODE"]

##甲减
hypothy_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2409','2461','2468'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['243','244']))]["ICD9_CODE"]

# 肾衰
renlfail_codes = d_icd_dia[(d_icd_dia.ICD9_CODE.isin(['40301','40311','40391','40402','40403','40412','40413','40492','40493'])) |
                           (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['5880','V420','V451'])) |
                           (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['585','586','V56']))]["ICD9_CODE"]

##肝病
liver_codes = d_icd_dia[(d_icd_dia.ICD9_CODE.isin(['07022','07023','07032','07033','07044','07054'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['0706','0709','4560','4561','4562','5722','5723','5724','5728','5733','5734','5738','5739','V427'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['570','571']))]["ICD9_CODE"]

   
##消化性溃疡
ulcer_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['5317','5319','5327','5329','5337','5339','5347','5349']))]["ICD9_CODE"]
   #ulcer     /* Chronic Peptic ulcer disease (includes bleeding only if obstruction is also present) */

##艾滋病
aids_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['042','043','044']))]["ICD9_CODE"]
   

##淋巴瘤
lymph_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2030','2386'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['200','201','202']))]["ICD9_CODE"]

##转移性肿瘤
mets_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['196','197','198','199']))]["ICD9_CODE"]

## 无转移实体瘤
tumor_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['140','141','142','143','144','145','146','147','148','149','150','151','152'
                                                                            ,'153','154','155','156','157','158','159','160','161','162','163','164','165'
                                                                            ,'166','167','168','169','170','171','172','174','175','176','177','178','179'
                                                                            ,'180','181','182','183','184','185','186','187','188','189','190','191','192'
                                                                            ,'193','194','195']))]["ICD9_CODE"]

##结缔组织病
arth_codes = d_icd_dia[(d_icd_dia.ICD9_CODE.isin(['72889','72930'])) |
                       (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['7010','7100','7101','7102','7103','7104','7108','7109','7112','7193','7285'])) |
                       (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['446','714','720','725']))]["ICD9_CODE"]

##凝血障碍
coag_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2871','2873','2874','2875'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['286']))]["ICD9_CODE"]

##肥胖
obese_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2780']))]["ICD9_CODE"]

##减肥？
wghtloss_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['7832','7994'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['260','261','262','263']))]["ICD9_CODE"]

##水电解质紊乱
lytes_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2536'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['276']))]["ICD9_CODE"]

##失血
bldloss_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2800']))]["ICD9_CODE"]

##贫血
amemdef_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2801','2808','2809'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['281']))]["ICD9_CODE"]

##酗酒
alcohol_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2652','2911','2912','2913','2915','2918','2919','3030','3039','3050','3575','4255','5353','5710','5711','5712','5713','V113'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['980']))]["ICD9_CODE"]

##药物滥用
drug_codes = d_icd_dia[(d_icd_dia.ICD9_CODE.isin(['V6542'])) |
                 (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['3052','3053','3054','3055','3056','3057','3058','3059'])) |
                 (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['292','304']))]["ICD9_CODE"]

##精神疾病
psych_codes = d_icd_dia[(d_icd_dia.ICD9_CODE.isin(['29604','29614','29644','29654'])) |
                        (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2938'])) |
                        (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['295','297','298']))]["ICD9_CODE"]

##抑郁症
depress_codes = d_icd_dia[(d_icd_dia["ICD9_CODE"].apply(lambda x:x[:4]).isin(['2962','2963','2965','3004'])) |
  (d_icd_dia["ICD9_CODE"].apply(lambda x:x[:3]).isin(['309','311']))]["ICD9_CODE"]

In [5]:
codes_list = [chf_codes,arrhy_codes,valve_codes,pulmcirc_codes,perivasc_codes,htn_codes,htncx_codes,para_codes,neuro_codes,chrnlung_codes,dm_codes,dmcx_codes,hypothy_codes,
              renlfail_codes,liver_codes,ulcer_codes,aids_codes,lymph_codes,mets_codes,tumor_codes,arth_codes,coag_codes,obese_codes,wghtloss_codes,lytes_codes,bldloss_codes,
              amemdef_codes,alcohol_codes,drug_codes,psych_codes,depress_codes]
column_name_list = ["chf","arrhy","valve","pulmcirc","perivasc","htn","htncx","para","neuro","chrnlung","dm","dmcx","hypothy",
                    "renlfail","liver","ulcer","aids","lymph","mets","tumor","arth","coag","obese","wghtloss","lytes","bldloss",
                    "amemdef","alcohol","drug","psych","depress"]

In [8]:
def AddComorbidity(comorbidity_codes,comorbidity_column_name):
    """
    codes:list of comorbidites icd9_id 
    #column_name:name of column in the admission table
    """
    global admission
    admission = pd.merge(admission,GetComorbidities(comorbidity_codes,comorbidity_column_name),on="HADM_ID")
    return admission

In [124]:
%%time
test = pd.DataFrame(dias.groupby("HADM_ID")["ICD9_CODE"],columns=["HADM_ID","ICD9_CODES"])

%%time
test[column_name_list[10]] = test["ICD9_CODES"].apply(lambda x:1.in(x.isin(codes_list[10])).map({False:0,True:1}).values)

%%time
for i in range(len(codes_list)):
    
    test[column_name_list[i]] = test["ICD9_CODES"].apply(lambda x:1.in(x.isin(codes_list[i])).map({False:0,True:1}).values)
    print(column_name_list[i]+" was added.")

test["HADM_ID"] = test["HADM_ID"].astype("int")

admission["HADM_ID"] = admission["HADM_ID"].astype("int")

admission_1 = pd.merge(admission,test,on="HADM_ID")

admission_1.to_csv("admission_with_comorbidity.csv")

In [193]:
admission_1.shape

(50796, 63)

In [188]:
chf = pd.DataFrame(dias.groupby("HADM_ID")["ICD9_CODE"],columns=["HADM_ID","ICD9_CODES"])

In [189]:
%%time
chf["chf"] = chf["ICD9_CODES"].apply(lambda x:1.in(x.isin(chf_codes).map({False:0,True:1}).values))

Wall time: 1min 21s


In [190]:
admission_2 = pd.merge(admission,chf,on="HADM_ID")

In [147]:
%%time
dia_sum = pd.DataFrame(dias.groupby("HADM_ID")["ICD9_CODE"],columns=["HADM_ID","ICD9_CODES"])

Wall time: 11.8 s


In [165]:
1 in(chf_codes.isin(dia_sum["ICD9_CODES"][0].values).map({False:0,True:1}).values)

False

In [180]:
admission_1.iloc[:,[30,33,62]].apply(lambda x:x.value_counts()).T

,0.0,1.0
chf_x,36252,14544
chf_y,36252,14544
depress,45608,5188
